In [ ]:
!pip install optuna

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [27]:
import os
import gc
import random
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,roc_auc_score



In [15]:
train = pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/test.csv')
columns = [col for col in train.columns.to_list() if col not in ['id','song_popularity']]
data=train[columns]
target=train['song_popularity']

In [18]:
NUM_CLASSES=target.nunique()

In [17]:
params={'alpha': 0.002165284049749127,
 'colsample_bytree': 0.41961032521264147,
 'eval_metric': 'mlogloss',
 'lambda': 0.06732506334605969,
 'learning_rate': 0.0718988345727017,
 'max_depth': 17,
 'min_child_weight': 158,
 'num_class': 2,
 'objective': 'multi:softprob',
 'subsample': 0.7270128671619758,
 'tree_method': 'gpu_hist'}

In [ ]:
# for i, (train_index, test_index) in enumerate(skf.split(X, y)):
#     print('[Fold %d/%d]' % (i + 1, kfold))
#     X_train, X_valid = X[train_index], X[test_index]
#     y_train, y_valid = y[train_index], y[test_index]
#     # Convert our data into XGBoost format
#     d_train = xgb.DMatrix(X_train, y_train)
#     d_valid = xgb.DMatrix(X_valid, y_valid)
#     d_test = xgb.DMatrix(test.values)
#     watchlist = [(d_train, 'train'), (d_valid, 'valid')]

#     # Train the model! We pass in a max of 1,600 rounds (with early stopping after 70)
#     # and the custom metric (maximize=True tells xgb that higher metric is better)
#     mdl = xgb.train(params, d_train, 1600, watchlist, early_stopping_rounds=70, feval=gini_xgb, maximize=True, verbose_eval=100)

#     print('[Fold %d/%d Prediciton:]' % (i + 1, kfold))
#     # Predict on our test data
#     p_test = mdl.predict(d_test, ntree_limit=mdl.best_ntree_limit)
#     sub['target'] += p_test/kfold

In [ ]:
# for train_idx, val_idx in cv.split(X, y):
#     gc.collect()
#     dval = xgboost.DMatrix(X[val_idx], label=y[val_idx])
#     booster: xgboost.Booster = xgboost.train(
#         params=params,
#         dtrain=make_trainset(train_idx),
#         num_boost_round=600,
#         evals=[(dval, 'val')],
#         early_stopping_rounds=30,
#         verbose_eval=100,
#     )
#     oof_proba[val_idx] = booster.predict(
#         dval, iteration_range=(0, booster.best_iteration + 1)
#     )
#     test_proba += booster.predict(
#         xgboost.DMatrix(X_test), iteration_range=(0, booster.best_iteration + 1)
#     ) / cv.n_splits

In [29]:
from tqdm import tqdm
np.random.seed(64)
random.seed(64)


target = train['song_popularity'].values
FOLDS=10
kf = StratifiedKFold(n_splits = FOLDS, shuffle=True, random_state=64)    
        
oof = np.zeros((train.shape[0],))
predictions = 0
auc_value = []
for f, (train_index, test_index) in tqdm(enumerate(kf.split(X=train, y=target))):
        X_train, X_val = train.iloc[train_index][columns], train.iloc[test_index][columns]
        y_train, y_val = target[train_index], target[test_index]
        
        model = xgb.XGBClassifier(**params)
        
        model.fit(
            X_train[columns], 
            y_train,
            eval_set=[(X_val[columns], y_val)],
            early_stopping_rounds=30,
            verbose=100
        )
        
        oof_temp_pred = model.predict_proba(X_val[columns])[:,1]
        test_temp_pred = model.predict_proba(test[columns])[:,1]
        oof[test_index] = oof_temp_pred
        predictions += test_temp_pred/FOLDS
        auc = roc_auc_score(y_val, oof_temp_pred)
        auc_value.append(auc)
        print(f'FOLD: {f} AUC: {auc} Mean AUC: {np.mean(auc_value)}')

0it [00:00, ?it/s]

[0]	validation_0-mlogloss:0.687724
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[66]	validation_0-mlogloss:0.646379



1it [00:01,  1.72s/it]

FOLD: 0 AUC: 0.5798917507005787 Mean AUC: 0.5798917507005787
[0]	validation_0-mlogloss:0.687549
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[46]	validation_0-mlogloss:0.650187



2it [00:03,  1.51s/it]

...

FOLD: 7 AUC: 0.575156304131739 Mean AUC: 0.5729198827011801
[0]	validation_0-mlogloss:0.687559
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[59]	validation_0-mlogloss:0.648648



9it [00:13,  1.53s/it]

FOLD: 8 AUC: 0.5711798978802213 Mean AUC: 0.572726551054407
[0]	validation_0-mlogloss:0.687301
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[68]	validation_0-mlogloss:0.644461



10it [00:15,  1.52s/it]

FOLD: 9 AUC: 0.5900304243982304 Mean AUC: 0.5744569383887892


In [34]:
submission_df =pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/sample_submission.csv')
submission_df['song_popularity'] = predictions
submission_df.to_csv('submission_xgb_best_params1_200_trials.csv', index=False)